This file constructs a new data set.

In [3]:
# Load dependencies
import datetime as dt
import numpy as np
import pandas as pd
from mosek.fusion import *
from tqdm import tqdm
import math
import gc
from sys import getsizeof
from tqdm import tqdm
from pathlib import Path

# Imports from module
from EITP.Models.TrackingModelSAA import TrackingModelSAA as TrackingModelSAA;
from EITP.Models.ExcessCVaRModelSAA import ExcessCVaRModelSAA as ExcessCVaRModelSAA;
from EITP.Models.TrackingModelDRO import TrackingModelDRO as TrackingModelDRO;
from EITP.Models.ExcessCVaRModelDRO import ExcessCVaRModelDRO as ExcessCVaRModelDRO;
from EITP.PerformanceEvaluation.QuantitativeStatistics import PerformanceMetrics;
from EITP.Data.DataLoader import DataLoader;
from EITP.Auxiliaries.Logger import write_parameters_to_file;

Then we load the data from the DataLoader.

In [4]:
# Specify first date
startDate = "2001-01-01"

# Start by instantiating the data loader
dataLoader = DataLoader(path='./Data/');
priceData = dataLoader.SP500(freq="daily", startDate=startDate, endDate=dt.datetime.today().strftime("%Y-%m-%d"));
yieldData = dataLoader.InterestRates(maturity="3 Mo", startDate=startDate, endDate=dt.datetime.today().strftime("%Y-%m-%d"))
totalObservations = priceData.shape[0]

# Convert to daily yields (only works for 3 Mo)
yieldData["3 Mo"] = (1 + yieldData['3 Mo']/100)**(1/252) - 1

# Get asset data
aggregateData = pd.merge(yieldData, priceData, on='Dates', how='inner')

Now the dates match. Then we calculate the returns separately and concatenate bond yields and equity returns.

In [5]:
# Format the output
dates = aggregateData['Dates']
index = aggregateData['SPX-INDEX']
indexReturns = np.array([None] + list(index.pct_change().dropna(axis=0)))
equity = aggregateData.drop(labels=['Dates', '3 Mo', 'SPX-INDEX'], axis=1)
equityReturns = equity.pct_change().dropna(axis=0)
treasuryReturns = aggregateData['3 Mo'].values[:-1].reshape(-1,1)
assetReturns = np.concatenate([treasuryReturns, equityReturns], axis=1)

# Add row of None to assetReturns
noneArray = np.array([None for i in range(assetReturns.shape[1])])
assetReturns = np.concatenate([noneArray.reshape(1,-1), assetReturns], axis=0)

# Save the data to avoid further mistakes
allReturns = pd.DataFrame(assetReturns)
allReturns.insert(0, 'Dates', dates)
allReturns.insert(1, 'SPX-INDEX', indexReturns)
allReturns = allReturns.rename(columns={0: 'TREASURY-3M'})

Save the returns into a file of returns on assets.

In [6]:
filepath = Path('./Data/Combined/dailyEquity_3MTreasury_Intersect.csv')
allReturns.to_csv(filepath, index=False)

In [7]:
allReturns

,Dates,SPX-INDEX,TREASURY-3M,1,2,3,4,5,6,7,...,253,254,255,256,257,258,259,260,261,262
0,2001-01-02,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2001-01-03,0.050099,0.000226,-0.029819,-0.055712,0.21304,-0.01069,0.160427,0.099375,-0.018213,...,-0.057577,0.074048,-0.064169,0.082638,0.303743,0.077308,0.103184,0.056332,-0.031522,-0.015326
2,2001-01-04,-0.010552,0.00022,-0.054885,-0.040174,0.0175,0.011851,0.059744,-0.008528,-0.07942,...,-0.071359,0.020488,-0.039514,-0.019608,-0.101118,-0.031284,0.037769,-0.008106,-0.017559,-0.058366
3,2001-01-05,-0.026242,0.000208,-0.007259,0.013633,-0.074631,-0.047882,-0.04589,-0.043005,-0.039673,...,0.013158,-0.053856,-0.018553,-0.017857,-0.136235,-0.048533,-0.054764,-0.010753,-0.019616,0.0
4,2001-01-08,-0.001918,0.000198,0.012869,-0.011798,-0.006306,-0.017728,-0.014654,-0.007789,0.016393,...,0.024935,-0.00842,0.006849,-0.013455,0.002291,-0.050815,-0.032873,0.061739,-0.003112,0.000992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5710,2023-11-13,-0.000836,0.000214,-0.010968,-0.016961,0.009552,-0.007314,-0.00385,-0.003895,0.002877,...,0.016816,0.009222,-0.014161,-0.003076,-0.028105,-0.000259,-0.009311,0.000411,-0.001569,0.00947
5711,2023-11-14,0.019075,0.000214,0.033269,0.009662,0.017083,0.031697,0.043577,0.057674,-0.035739,...,0.016127,0.020451,0.032521,0.022299,0.04157,0.016652,0.038619,0.004936,0.012114,0.026581
5712,2023-11-15,0.001597,0.000213,0.000552,0.031784,-0.017571,0.024561,0.017495,0.016636,-0.016394,...,-0.010716,-0.000906,-0.002671,-0.001338,0.006941,0.01606,0.01783,0.025213,0.00717,0.0022
5713,2023-11-16,0.00119,0.000214,0.006405,-0.013912,0.005348,0.004102,-0.000251,-0.008182,0.009857,...,0.006703,-0.000213,0.017799,0.008184,-0.010832,-0.002383,0.005194,-0.003274,0.008605,-0.025836


In [1]:
"""
Author: Andreas Heidelbach Engly (s170303)
Purpose: This file generates the results for Wasserstein-based DRO of ExcessCVaR model used in the thesis.

Inputs:
- In order to get the right results, the model parameters must be modified (look e.g. at ExperimentLog.txt)).

Outputs:
- The results are saved into .csv-files. They are reshaped into 1D-arrays and need to be recovered with dimension specified in ExperimentLog.txt.
"""

# Load dependencies
import datetime as dt
import numpy as np
from mosek.fusion import *
from tqdm import tqdm
import math
import gc
from sys import getsizeof
from tqdm import tqdm

# Imports from module
from EITP.Models.TrackingModelSAA import TrackingModelSAA as TrackingModelSAA;
from EITP.Models.ExcessCVaRModelSAA import ExcessCVaRModelSAA as ExcessCVaRModelSAA;
from EITP.Models.TrackingModelDRO import TrackingModelDRO as TrackingModelDRO;
from EITP.Models.ExcessCVaRModelDRO import ExcessCVaRModelDRO as ExcessCVaRModelDRO;
from EITP.PerformanceEvaluation.QuantitativeStatistics import PerformanceMetrics;
from EITP.Data.DataLoader import DataLoader;
from EITP.Auxiliaries.Logger import write_parameters_to_file;

# Print that script is starting
print("\n#########################################################################################\n")
print("                           Reproducing Results for Chapter 5                                 ")
print("\n#########################################################################################\n")

#########################################################################################
#                             Loading Market Data
#########################################################################################

# Start by instantiating the data loader
dataLoader = DataLoader(path='./Data/');
priceData = dataLoader.SP500(freq="daily", startDate="2012-01-01", endDate=dt.datetime.today().strftime("%Y-%m-%d"));
totalObservations = priceData.shape[0]

# Format the output
dates = priceData.iloc[:,0]
index = priceData.iloc[:,1]
indexReturns = priceData.iloc[:,1].pct_change().dropna(axis=0).values
assets = priceData.iloc[:,2:]
assetsReturns = priceData.iloc[:,2:].pct_change().dropna(axis=0).values

# Then we can free priceData from the memory
del dataLoader
del priceData
gc.collect()

#########################################################################################
#                             Control Experiments
#########################################################################################

# Specify whether to run experiments or not
runTests = True
runExperiment1 = False
runExperiment2 = False

# Specify number of simulations in each iteration of all experiments
nSimulations = 200

# Specify central parameters
rho = 2
beta = 0.90
excessReturnAnually = 0
rfAnnualy = 0.02

#########################################################################################
#                             Test Runs on Both Models
#########################################################################################

if runTests:

    # Set model parameters
    totalEps = 30
    startEps = -6
    endEps = 0.5
    epsCollection = np.concatenate(([0], 10**np.linspace(startEps, endEps, totalEps)), axis=0)
    alphaDaily = (1 + excessReturnAnually)**(1/252)-1
    rfDaily = (1 + rfAnnualy)**(1/252)-1

    # Run a simple test
    startIndex = 230
    endIndex = startIndex + 700
    print("\nTesting SAA:\n")
    modelSAA = ExcessCVaRModelSAA(returnsAssets=assetsReturns[startIndex:endIndex,:],
                                returnsIndex=indexReturns[startIndex:endIndex],
                                beta=beta,
                                rho=rho,
                                alpha=alphaDaily,
                                rf=rfDaily)
    resultsSAA = modelSAA.solve(rhoCollection=np.array([rho]),
                betaCollection=np.array([beta]), progressBar=True)
    print(resultsSAA)
    print("\nTesting DRO:\n")

    modelDRO = ExcessCVaRModelDRO(returnsAssets=assetsReturns[startIndex:endIndex,:],
                                returnsIndex=indexReturns[startIndex:endIndex],
                                beta=beta,
                                rho=rho,
                                alpha=alphaDaily,
                                rf=rfDaily)
    resultsDRO = modelDRO.solve(epsCollection=epsCollection, rhoCollection=np.array([rho]),
                betaCollection=np.array([beta]), progressBar=True)
    print(resultsDRO)

    # Remove MODEL from memory
    del modelSAA
    del modelDRO
    del resultsSAA
    del resultsDRO
    gc.collect()


#########################################################################################

                           Reproducing Results for Chapter 5                                 

#########################################################################################


Testing SAA:



100%|█████████████████████████████████████████| 1/1 [00:00<00:00,  6.79it/s]


        obj  rho beta excessReturns       VaR      CVaR    1        2  \
0  0.000129  2.0  0.9      0.000297  0.000168  0.000213  0.0  0.00262   

          3        4  ...       304       305  306  307       308       309  \
0  0.014021  0.00554  ...  0.006512  0.001984  0.0  0.0  0.004592  0.001903   

   310       311  312  313  
0  0.0  0.006712  0.0  0.0  

[1 rows x 319 columns]

Testing DRO:



100%|███████████████████████████████████████| 31/31 [00:06<00:00,  4.71it/s]

         obj       eps  rho beta excessReturns       VaR      CVaR         1  \
0   0.000129       0.0  2.0  0.9      0.000297  0.000168  0.000213       0.0   
1    0.00013  0.000001  2.0  0.9      0.000297  0.000169  0.000213       0.0   
2    0.00013  0.000002  2.0  0.9      0.000297  0.000169  0.000213       0.0   
3   0.000131  0.000003  2.0  0.9      0.000297  0.000169  0.000213       0.0   
4   0.000133  0.000005  2.0  0.9      0.000298  0.000168  0.000213       0.0   
5   0.000135  0.000008  2.0  0.9      0.000298  0.000168  0.000213       0.0   
6   0.000139  0.000013  2.0  0.9      0.000298  0.000169  0.000214       0.0   
7   0.000145  0.000022  2.0  0.9      0.000298   0.00017  0.000214       0.0   
8   0.000156  0.000037  2.0  0.9      0.000299   0.00017  0.000214       0.0   
9   0.000174  0.000062  2.0  0.9        0.0003  0.000165  0.000215       0.0   
10  0.000204  0.000104  2.0  0.9      0.000299  0.000164  0.000215       0.0   
11  0.000252  0.000174  2.0  0.9      0.